In [1]:
# this chemkin file is from the cti generated by rmg

# load a bunch of stuff
from __future__ import division
# load
import cantera as ct
import numpy as np
import scipy
import pylab
import matplotlib
import matplotlib.pyplot  as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import NullFormatter, MaxNLocator, LogLocator
import csv
from pydas.dassl import DASSL
import os
import rmgpy
import rmg
import re
import operator
import pandas as pd
%matplotlib inline

gas = ct.Solution('./chem.cti','gas')
surf = ct.Interface('./chem.cti','surface1', [gas])
print "This mechanism contains %d gas reactions and %d surface reactions"%(gas.n_reactions, surf.n_reactions)
i_n2 = gas.species_index('N2')
i_ch4 = gas.species_index('CH4(2)')
i_o2 = gas.species_index('O2(3)')
i_co2 = gas.species_index('CO2(4)')
i_h2o = gas.species_index('H2O(5)')
i_h2 = gas.species_index('H2(6)')
i_co = gas.species_index('CO(7)')

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


/home/mazeau.e/Code/RMG-Py/rmgpy/tools/plot.py:34: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/mazeau.e/anaconda2/envs/rmg_env2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/mazeau.e/anaconda2/envs/rmg_env2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/mazeau.e/anaconda2/envs/rmg_env2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/mazeau.e/anaconda2/envs/rmg_env2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/mazeau.e/anaconda2/envs/rmg_env2/li

This mechanism contains 164 gas reactions and 144 surface reactions


  1.000000e-04    0.099996    0.100000    0.800003
  2.238721e-04    0.099996    0.100000    0.800003
  5.011872e-04    0.099996    0.100000    0.800003
  1.122018e-03    0.099996    0.099999    0.800003
  2.511886e-03    0.099995    0.099997    0.800003
  5.623413e-03    0.099992    0.099992    0.800003
  1.258925e-02    0.099987    0.099981    0.800003
  2.818383e-02    0.099975    0.099957    0.800003
  6.309573e-02    0.099948    0.099904    0.800003
  1.412538e-01    0.099888    0.099784    0.800003
  3.162278e-01    0.099753    0.099514    0.800003
  7.079458e-01    0.099451    0.098910    0.800003
  1.584893e+00    0.098772    0.097554    0.800004
  3.548134e+00    0.097252    0.094515    0.800005
  7.943282e+00    0.093873    0.087761    0.800007
  1.778279e+01    0.086644    0.073313    0.800014
  3.981072e+01    0.072702    0.045447    0.800026
  8.912509e+01    0.050360    0.000753    0.800027
  1.995262e+02    0.049139    0.000000    0.799576
  4.466836e+02    0.048701    0

In [2]:
#######################################################################
# Input Parameters
#######################################################################

volume = 1.0
cat_area_per_vol = 1.0E2 #I made this up!

cat_area = cat_area_per_vol * volume

#initial conditions
T0 = 1000.
P0 = 1.0 #bar
#initial composition (gas mol fraction)
CH4_0 = 0.1
O2_0 = 0.1
N2_0 = 0.8
CO2_0 = 0.

gas.TPX = T0, P0 * ct.one_atm, {'CH4(2)':CH4_0, 'O2(3)':O2_0, 'N2':N2_0, 'CO2(4)':CO2_0}
          #Temp, Pressure, mole fraction
surf.TP = T0, P0 * ct.one_atm
surf.coverages = {'X(1)':1.0}
#surf.coverages = {'PT(S)':0.9, 'O(S)':0.1}

r = ct.IdealGasConstPressureReactor(gas, energy='off')
r.volume = volume
rsurf = ct.ReactorSurface(surf, r, A=cat_area) # A=cat_area

sim = ct.ReactorNet([r])
sim.max_err_test_fails = 12

# set relative and absolute tolerances on the simulation
sim.rtol = 1.0e-12
sim.atol = 1.0e-18

#rxn_time = np.linspace(1E-12, 5, 10001) #
rxn_time = np.logspace(-4, 3, 10001) #from 0s to 1000s, log spacing

gas_mole_fracs = np.zeros([gas.n_species, len(rxn_time)])
surf_site_fracs = np.zeros([surf.n_species, len(rxn_time)])
temperature = np.zeros(len(rxn_time))    
pressure = np.zeros(len(rxn_time))   

for i in range(len(rxn_time)):
    time = rxn_time[i] #define time in the reactor
    sim.advance(time) #Advance the simulation to next set time
    temperature[i] = gas.T
    pressure[i] = gas.P/ct.one_atm
    gas_mole_fracs[:,i] = gas.X #
    surf_site_fracs[:,i] = surf.coverages #
    if not i % 500:
        print('  {0:10e}  {1:10f}  {2:10f}  {3:10f}'.format(time, *gas['CH4(2)','O2(3)','N2'].X))



#sim.advance_to_steady_state()
#print('  {0:10e}  {1:10f}  {2:10f}  {3:10f}'.format(sim.time, *gas['CH4','H2','CO'].X))    
    
# CH4_ref = gas_mole_fracs[i_ch4,:]

# setting new sensitivity to be based on time to consume 95% of the o2
zipped = zip(gas_mole_fracs[i_o2,:],rxn_time)
zipped = [list(z) for z in zipped]
for z in zipped:
    if z[0] <= O2_0 * .05:
        O2_ref_time = z[1]
        break

#Plot out simulations results
fig = pylab.figure(dpi=300,figsize=(8,8))
gs = gridspec.GridSpec(2, 1)
ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1])

y_min = 1E-3

for i in range(gas.n_species):
    if i != i_n2:
        if np.max(gas_mole_fracs[i,:]) > y_min:
            ax0.loglog(rxn_time, gas_mole_fracs[i,:], label=gas.species_name(i) )

for i in range(surf.n_species):
    if np.max(surf_site_fracs[i,:]) > y_min:
        ax1.loglog(rxn_time, surf_site_fracs[i,:], label=surf.species_name(i) )
            
ax0.legend(loc='lower left', fontsize = 12)
ax1.legend(loc='lower left', fontsize = 12)

ax0.set_ylabel("gas-phase mole fraction")
ax0.set_ylim(y_min,0.5)
ax0.set_xlim(1.0E-4,max(rxn_time))
ax1.set_xlim(1.0E-4,max(rxn_time))
ax0.set_xlabel("Time(s)")
ax1.set_ylabel("surface site fraction")
#ax0.xaxis.set_major_locator(MaxNLocator(6))
#ax0.yaxis.set_major_locator(LogLocator(base=10.0, numticks=3))
#ax0.tick_params(axis='both', which='major', labelsize=10)
ax1.set_ylim(y_min, 1.1 )
ax1.set_xlabel("Time(s)")


#######################
##   FLUX DIAGRAMS   ##
#######################
steps = len(temperature)
for i in range(0, steps, steps//20):
    gas.TP = temperature[i], pressure[i] * ct.one_atm
    gas.X  = gas_mole_fracs[:,i]
    surf.TP = temperature[i], pressure[i] * ct.one_atm
    surf.coverages = surf_site_fracs[:,i]
    
    element = 'O'

    diagram = ct.ReactionPathDiagram(surf, element)
    diagram.title = 'Reaction path diagram following {0}'.format(element)
    diagram.label_threshold = 0.000001

    dot_file = 'rxnpathO{:06d}.dot'.format(i)
    img_file = 'rxnpathO{:06d}.png'.format(i)
    img_path = os.path.join(os.getcwd(), img_file)

    diagram.write_dot(dot_file)
#     print(diagram.get_data())

    print("Wrote graphviz input file to '{0}'.".format(os.path.join(os.getcwd(), dot_file)))

    os.system('dot {0} -Tpng -o{1} -Gdpi=200'.format(dot_file, img_file))
    print("Wrote graphviz output file to '{0}'.".format(img_path))

    from IPython.display import Image
    Image(filename=img_file) 
    
for i in range(0, steps, steps//20):
    gas.TP = temperature[i], pressure[i] * ct.one_atm
    gas.X  = gas_mole_fracs[:,i]
    surf.TP = temperature[i], pressure[i] * ct.one_atm
    surf.coverages = surf_site_fracs[:,i]
    
    element = 'C'

    diagram = ct.ReactionPathDiagram(surf, element)
    diagram.title = 'Reaction path diagram following {0}'.format(element)
    diagram.label_threshold = 0.000001

    dot_file = 'rxnpathC{:06d}.dot'.format(i)
    img_file = 'rxnpathC{:06d}.png'.format(i)
    img_path = os.path.join(os.getcwd(), img_file)

    diagram.write_dot(dot_file)
#     print(diagram.get_data())

    print("Wrote graphviz input file to '{0}'.".format(os.path.join(os.getcwd(), dot_file)))

    os.system('dot {0} -Tpng -o{1} -Gdpi=200'.format(dot_file, img_file))
    print("Wrote graphviz output file to '{0}'.".format(img_path))

    from IPython.display import Image
    Image(filename=img_file) 


In [3]:
with open('time.txt','w') as f:
    f.write(str(O2_ref_time))

In [4]:
# Set the value of the perturbation
dk = 1.0e-2
# create the array to hold the sensitivty at each time step
Sensitivity = np.zeros([surf.n_reactions,len(rxn_time)])

rxns = []
sens = []

#cycle through each reaction
for m in range(surf.n_reactions):
    surf.set_multiplier(1.0) # reset all multipliers  
    surf.set_multiplier(1+dk, m) # perturb reaction m
    
    gas.TPX = T0, P0 * ct.one_atm, {'CH4(2)':CH4_0, 'O2(3)':O2_0, 'N2':N2_0}
              #Temp, Pressure, mole fraction
    surf.TP = T0, P0 * ct.one_atm
    surf.coverages = {'X(1)':1.0}
    
    r = ct.IdealGasConstPressureReactor(gas, energy='off')
    r.volume = volume
    rsurf = ct.ReactorSurface(surf, r, A=cat_area) # A=cat_area


    sim = ct.ReactorNet([r])
    sim.max_err_test_fails = 12


    # set relative and absolute tolerances on the simulation
    sim.rtol = 1.0e-12
    sim.atol = 1.0e-18
    gas_mole_fracs = np.zeros([gas.n_species, len(rxn_time)])
    temperature = np.zeros(len(rxn_time))    
    pressure = np.zeros(len(rxn_time))   

    for i in range(len(rxn_time)):
        time = rxn_time[i] #define time in the reactor
        sim.advance(time) #Advance the simulation to next set time
        temperature[i] = gas.T
        pressure[i] = gas.P/ct.one_atm
        gas_mole_fracs[:,i] = gas.X #[mol.frac,nth reactor]
    # here is the new output
#     CH4_new = gas_mole_fracs[i_ch4,:]

    O2_new = gas_mole_fracs[i_o2,:]
    zipped = zip(gas_mole_fracs[i_o2,:],rxn_time)
    zipped = [list(z) for z in zipped]
    for z in zipped:
        if z[0] <= O2_0 * .05:
            O2_new_time = z[1]
            break

#     Sens = (CH4_new-CH4_ref)/(CH4_ref*dk)
    Sens = (O2_ref_time - O2_new_time)/(O2_ref_time*dk) # positive if it speeds up the reaction
#     max_sensitivity = max(max(Sens),-min(Sens))
#    if (1==0):
#        if (max_sensitivity>0.01):
#            print "%d %s %.2F"%( m, surf.reaction_equations()[m],  max_sensitivity)
#    else:
#     print "%d %s %.2F"%( m, surf.reaction_equations()[m],  max_sensitivity)
    print "%d %s %.2F"%( m, surf.reaction_equations()[m],  Sens)
#     Sensitivity[m,:] = Sens
    rxns.append(surf.reaction_equations()[m])
    sens.append(Sens)
surf.set_multiplier(1.0)

 

0 CH4(2) + X(1) <=> CH4X(45) 0.00
1 O2(3) + 2 X(1) <=> 2 OX(20) 0.00
2 C2H6(8) + 2 X(1) <=> 2 CH3X(21) 0.00
3 CH3(10) + X(1) <=> CH3X(21) 0.00
4 H2(6) + 2 X(1) <=> 2 HX(19) 0.00
5 CH3X(21) + HX(19) <=> CH4(2) + 2 X(1) 0.64
6 H(12) + X(1) <=> HX(19) 0.00
7 CH2X(27) + HX(19) <=> CH3X(21) + X(1) 0.32
8 CHX(28) + HX(19) <=> CH2X(27) + X(1) 0.00
9 2 CH2X(27) <=> CH3X(21) + CHX(28) 0.00
10 CHX(28) + X(1) <=> CX(26) + HX(19) 0.00
11 CX(26) + H2(6) <=> CH2X(27) 0.00
12 CH3X(21) + CX(26) <=> CH2X(27) + CHX(28) 0.00
13 CH2X(27) + CX(26) <=> 2 CHX(28) 0.00
14 CH4(2) + OX(20) + X(1) <=> CH3X(21) + HOX(22) 0.00
15 HX(19) + OX(20) <=> HOX(22) + X(1) 0.00
16 H2O(5) + 2 X(1) <=> HOX(22) + HX(19) 0.00
17 CH3OH(14) + 2 X(1) <=> CH3X(21) + HOX(22) -0.16
18 OH(17) + X(1) <=> HOX(22) 0.32
19 CH2X(27) + HOX(22) <=> CH3X(21) + OX(20) 0.00
20 CHX(28) + HOX(22) <=> CH2X(27) + OX(20) 0.00
21 CX(26) + HOX(22) <=> CHX(28) + OX(20) 0.00
22 CH4(2) + HOX(22) + X(1) <=> CH3X(21) + H2OX(23) 0.16
23 H2O(5) + X(1) <=> H

In [5]:
#Plot out simulations results
# fig = pylab.figure(dpi=300,figsize=(8,8))
# gs = gridspec.GridSpec(1, 1)
# ax0 = plt.subplot(gs[0])


# for m in range(surf.n_reactions):
#     y_max = 0.00001
#     if (1==1):
#         if max(Sensitivity[m,:])>y_max or min(Sensitivity[m,:])<-y_max:
#             lab = surf.reaction_equations()[m]
#             ax0.semilogx(rxn_time, Sensitivity[m,:], label=lab )
#     else:
#         if max(Normed_sensitivity[m,:])>y_max or min(Normed_sensitivity[m,:])<-y_max:
#             lab = surf.reaction_equations()[m]
#             ax0.semilogx(rxn_time, Normed_sensitivity[m,:], label=lab )
        
# ax0.legend(loc='best', fontsize = 12)   
# ax0.set_xlim(rxn_time[1], rxn_time[-1] )

# ax0.set_xlabel("Time(s)")
# ax0.set_ylabel("Sensitivity Coefficient")

In [6]:
# # now integrate with respect to time
# from numpy import trapz

# # for m in range(surf.n_reactions):
# #     ans = trapz(Sensitivity[m,:], rxn_time)#/max(rxn_time)
# #     if ans>1E-4:
# #         print "%d %s %.5f"%( m, surf.reaction_equations()[m], ans )

# n = []
# rxns = []
# for m in range(surf.n_reactions):
#     n.append(trapz(Sensitivity[m,:], rxn_time))#/max(rxn_time)
#     rxns.append(surf.reaction_equations()[m])

# ans = dict(zip(rxns,n))
# sorted_ans = sorted(ans.items(), key=operator.itemgetter(1), reverse=True)

# for x in sorted_ans:
#     print '%s %.4f'%(x)

In [7]:
species_dict = rmgpy.data.kinetics.KineticsLibrary().getSpecies('species_dictionary.txt')
keys = species_dict.keys()

# get the first listed smiles string for each molecule
smile = []
for s in species_dict:
    smile.append(species_dict[s].molecule[0])
    if len(species_dict[s].molecule) is not 1:
        print 'There are %d dupllicate smiles for %s:'%(len(species_dict[s].molecule),s)
        for a in range(len(species_dict[s].molecule)):
            print '%s'%(species_dict[s].molecule[a])
            
# translate the molecules from above into just smiles strings
smiles = []
for s in smile:
    smiles.append(s.toSMILES())
    
names = dict(zip(keys,smiles))

rxns_translated = []

# now replace each molecule with smiles
for x in rxns:
    for key, smile in names.iteritems():
        x = re.sub(re.escape(key), smile, x)
    rxns_translated.append(x)

answer = dict(zip(rxns_translated, sens))
sorted_answer = sorted(answer.items(), key=operator.itemgetter(1), reverse=True)
# for x in sorted_answer:
#     print '%s %.4f'%(x)

There are 2 dupllicate smiles for CHO3(116):
<Molecule "[O]C(=O)O">
<Molecule "[O]C(=O)O">


In [8]:
# write to csv file
(pd.DataFrame.from_dict(data=sorted_answer, orient='columns')
.to_csv('dict.csv', header=False))
